In [1]:
from helper_functions import *
import nengo
import nengo_dl

# Specifying architecture in Tensorflow

In [2]:
inp = tf.keras.Input(shape=(2))
dense1 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(dense1)
output = layers.Dense(1, use_bias = True)(dense2)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

# Loading weighs from .onnx file

In [3]:
onnx_model_path = '../benchmarks/controller_single_pendulum.onnx'
onnx_model = onnx.load(onnx_model_path)
params_array = []

for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    params_array.append(tensor_array)
    
weights_list=[]
bias_list=[]
i=0
for tensor_array in params_array:
    if(i%2==0):
        weights_list.append(tensor_array)
    i = i+1
i=0
for tensor_array in params_array:
    if(i%2==1):
        bias_list.append(tensor_array)
    i = i+1

weights_list.reverse()
bias_list.reverse()

for i in range(1, len(weights_list) + 1):
    transposed_weights = weights_list[i-1]
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

# Extracting weights from the .onnx file

In [4]:
weights, biases = extract_model_params_tf(nn_ctlr)

# Function to check the upper bound property

In [5]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    # Constraining input constraints
    for num in range(1,inputs+1):
        declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

    # For timestep 0,
    # Initializing variables for stored potentials of all layers except last
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")
    
    # Stored potential for the last layer
    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    # Initializing other neuron variables for the hidden layers
    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")

    # Initializing only the instant potential value for the output layer
    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")

    
    # Encodings
    # Potentials for all neurons initialized to zero for timestep 1 
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")
    
    
    thresh = 1
    lamb = 1
    M = 999999
    eps = 0.00001
    
    # Encodings for the SRLA activation
    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                # For the first hidden layer, the repeating input is multiplied with the weights
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                # For all other layers, weights are multiplied to the amplituides of the neuron in the previous layer
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                
    # Output is calculated as the sum of instant potentials at the output neuron(s)
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        # Calculation of instant potential at the final neuron
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    
    # Encoding of the verification query - negation of the property
    equations.append(f"model.addConstr({out}>={cond[1]*time_steps})")
    # The objective function is set to 'MAXIMIZE' in order to provide a heuristic to the solver for constraint solving
    equations.append(f'model.setObjective({out}, gp.GRB.MAXIMIZE)')


    return equations, declare

# Function to check the lower bound property

In [6]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    for num in range(1,inputs+1):
        declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")

    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 999999
    eps = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                

    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}<={cond[0]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MINIMIZE)')

    return equations, declare

# Function to set parameters and solve SNN encodings with Gurobi

In [7]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_SP_sim_random.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
    model=gp.Model("Encodings")

    model.Params.MIPFocus = focus
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1
    try:
        f = open(file_path,"r")
        try:
            for l in f:
                exec(l)
        finally:
            f.close()
    except IOError:
        pass
    model.optimize()
    
    return model

## SNN specifications
### Needs to be changed for different benchmarks

In [8]:
layer_no = 3
neuron_no = 1
time_steps = 5
input_bounds = [[1.0,1.2],[0.0,0.2]]
output_range = [-0.781295 ,-0.542820]

# Range verification query for increasing NUMSTEPS

In [16]:
for NUMS in range(1,6):
    print('Checking with NUMSTEPS ',NUMS)
    print('Checking LB', end=':\t')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n')

    print('Checking UB', end=':\t')
    # The line below generates the entire SNN encoding as strings and stores them into the variables equations and declare
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    # The line below uses Gurobi solver for constraint solving
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n\n')
print("\n\n")

Checking with NUMSTEPS  1
Checking LB:	Property holds 3
Runtime 0.006000041961669922
Checking UB:	Property does not hold
Runtime:  0.003000020980834961

Checking with NUMSTEPS  2
Checking LB:	Property holds 3
Runtime 0.006999969482421875
Checking UB:	Property does not hold
Runtime:  0.009999990463256836

Checking with NUMSTEPS  3
Checking LB:	Property holds 3
Runtime 0.010999917984008789
Checking UB:	Property does not hold
Runtime:  0.01399993896484375

Checking with NUMSTEPS  4
Checking LB:	Property holds 3
Runtime 0.015000104904174805
Checking UB:	Property does not hold
Runtime:  0.053999900817871094

Checking with NUMSTEPS  5
Checking LB:	Property holds 3
Runtime 0.03299999237060547
Checking UB:	Property does not hold
Runtime:  0.039999961853027344






# Range verification query for increasing NUMSTEPS
### (a) with safe range bounds loosened by 0.1

In [17]:
change = 0.1
output_range = [-0.781295 - change, -0.542820 + change]
for NUMS in range(1,6):
    print('Checking with NUMSTEPS ',NUMS)
    print('Checking LB', end=':\t')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n')

    print('Checking UB', end=':\t')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n\n')
print("\n\n")

Checking with NUMSTEPS  1
Checking LB:	Property holds 3
Runtime:  0.003999948501586914
Checking UB:	Property does not hold
Runtime:  0.003999948501586914

Checking with NUMSTEPS  2
Checking LB:	Property holds 3
Runtime:  0.006999969482421875
Checking UB:	Property does not hold
Runtime:  0.00800013542175293

Checking with NUMSTEPS  3
Checking LB:	Property holds 3
Runtime:  0.009999990463256836
Checking UB:	Property does not hold
Runtime:  0.013000011444091797

Checking with NUMSTEPS  4
Checking LB:	Property holds 3
Runtime:  0.015000104904174805
Checking UB:	Property does not hold
Runtime:  0.02499985694885254

Checking with NUMSTEPS  5
Checking LB:	Property holds 3
Runtime:  0.029999971389770508
Checking UB:	Property does not hold
Runtime:  0.034999847412109375






### (b) with safe range bounds loosened by 0.2

In [20]:
change = 0.2
output_range = [-0.781295 - change, -0.542820 + change]
for NUMS in range(1,7):
    print('Checking with NUMSTEPS ',NUMS)
    print('Checking LB', end=':\t')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n')

    print('Checking UB', end=':\t')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n\n')
print("\n\n")

Checking with NUMSTEPS  1
Checking LB:	Property holds 3
Runtime:  0.004000186920166016
Checking UB:	Property does not hold
Runtime:  0.003000020980834961

Checking with NUMSTEPS  2
Checking LB:	Property holds 3
Runtime:  0.009000062942504883
Checking UB:	Property does not hold
Runtime:  0.00800013542175293

Checking with NUMSTEPS  3
Checking LB:	Property holds 3
Runtime:  0.010999917984008789
Checking UB:	Property does not hold
Runtime:  0.010999917984008789

Checking with NUMSTEPS  4
Checking LB:	Property holds 3
Runtime:  0.014000177383422852
Checking UB:	Property does not hold
Runtime:  0.021000146865844727

Checking with NUMSTEPS  5
Checking LB:	Property holds 3
Runtime:  0.03399991989135742
Checking UB:	Property holds 4
Runtime:  0.03600001335144043

Checking with NUMSTEPS  6
Checking LB:	Property holds 3
Runtime:  0.0409998893737793
Checking UB:	Property holds 4
Runtime:  0.0409998893737793






### (c) with safe range bounds loosened by 0.4

In [22]:
change = 0.4
output_range = [-0.781295 - change, -0.542820 + change]
for NUMS in range(1,7):
    print('Checking with NUMSTEPS ',NUMS)
    print('Checking LB', end=':\t')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n')

    print('Checking UB', end=':\t')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print('Property holds', model3.status)
    print('Runtime: ',model3.Runtime,end='\n\n')
print("\n\n")

Checking with NUMSTEPS  1
Checking LB:	Property holds 3
Runtime:  0.003999948501586914
Checking UB:	Property does not hold
Runtime:  0.003000020980834961

Checking with NUMSTEPS  2
Checking LB:	Property holds 3
Runtime:  0.006999969482421875
Checking UB:	Property does not hold
Runtime:  0.006999969482421875

Checking with NUMSTEPS  3
Checking LB:	Property holds 3
Runtime:  0.01100015640258789
Checking UB:	Property holds 4
Runtime:  0.010999917984008789

Checking with NUMSTEPS  4
Checking LB:	Property holds 3
Runtime:  0.02200007438659668
Checking UB:	Property holds 4
Runtime:  0.01900005340576172

Checking with NUMSTEPS  5
Checking LB:	Property holds 3
Runtime:  0.03400015830993652
Checking UB:	Property holds 4
Runtime:  0.02500009536743164

Checking with NUMSTEPS  6
Checking LB:	Property holds 3
Runtime:  0.0409998893737793
Checking UB:	Property holds 4
Runtime:  0.03699994087219238






# Bounds Tightening

We found that the lower bounds of the Single Pendulum were not violated. Therefore, we only tighten its upper bounds. 

For this implementation, we have run 500 random samples from the input space. The lowest observed values have been taken and stored in to the folowing array 

In [25]:
highest_out = [0,0,0,0,-0.294831, -0.360207 , -0.425026, -0.484878,-0.424881,-0.446818, -0.500576, -0.490154, -0.464801, -0.469808, -0.459036, -0.478375, -0.473631, -0.490891,-0.513279, -0.528757, -0.510519]

# Binary Search for Bound Tightening

In [43]:
change_bound = 0.2 # Enter the value you want to add to the UB specification to widen it
T_up = 6
for NUMS in range(5, T_up + 1): # Loop runs from 5 since the SNNs with below 5 NUMSTEPS were not accurate enough
    count = 1
    runs_for = 0
    print('NUMSTEPS:',NUMS,"\n")
    output_range = [0,-0.542820 + change_bound] # Since the UB was found to be within 0.2 of the given safety bounds
    flag = 0
    lb = -0.542820
    ub = -0.542820  + change_bound
    checked = []
    result = []
    while (flag <= 1):
        print("Iteration",count)
        count = count + 1
        print("LB:", lb)
        print("UB:", ub)
        mid = (lb+ub)/2
#         mid = random.uniform(lb, ub)
        if(len(checked) > 1 and abs(ub - lb) < 0.001) :
            flag = flag + 1

        output_range[1] = mid
        print("Checking with UB:", output_range[1])
        
        # The code below implements random simulations in an attempt to violate the output range
        if(highest_out[NUMS] > mid):
            print('Result: Violated by 500 random simulations')
            lb = mid
            result.append('X')
            checked.append(output_range[1])
            print("Previous Results:",result,'\n')
            continue
        
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,NUMS,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        checked.append(output_range[1])
        if(model3.status in [2,10]):
            print('Result: Property does not hold')
            lb = mid 
        elif(model3.status in [9]):
            print(NUMS, 'Time out')
            break
        else:
            print('Result: Property holds',model3.status)
            ub = mid
        result.append(model3.status)
        print('Runtime',model3.Runtime) 
        runs_for = runs_for + model3.Runtime
        print(checked)
        print("Previous Results:",result,'\n')
        
    print("\nTotal time taken:", runs_for)
    print("Upper Bound of SNN output:", ub, "\n\n")

NUMSTEPS: 5 

Iteration 1
LB: -0.54282
UB: -0.34281999999999996
Checking with UB: -0.44282
Result: Violated by 500 random simulations
Previous Results: ['X'] 

Iteration 2
LB: -0.44282
UB: -0.34281999999999996
Checking with UB: -0.39281999999999995
Result: Violated by 500 random simulations
Previous Results: ['X', 'X'] 

Iteration 3
LB: -0.39281999999999995
UB: -0.34281999999999996
Checking with UB: -0.3678199999999999
Result: Violated by 500 random simulations
Previous Results: ['X', 'X', 'X'] 

Iteration 4
LB: -0.3678199999999999
UB: -0.34281999999999996
Checking with UB: -0.35531999999999997
Result: Property holds 4
Runtime 0.029000043869018555
[-0.44282, -0.39281999999999995, -0.3678199999999999, -0.35531999999999997]
Previous Results: ['X', 'X', 'X', 4] 

Iteration 5
LB: -0.3678199999999999
UB: -0.35531999999999997
Checking with UB: -0.36156999999999995
Result: Violated by 500 random simulations
Previous Results: ['X', 'X', 'X', 4, 'X'] 

Iteration 6
LB: -0.36156999999999995
UB: -